In [10]:
!pip install llama-index transformers accelerate bitsandbytes chromadb sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=64ca18205001ae9cd898254806e5e7114fe858a84ce9f2083f640ecded285c97
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [31]:
import importlib
import factsheet_generator
importlib.reload(factsheet_generator)
from factsheet_generator import FactsheetGenerator, load_local_models
from chromadb.utils import embedding_functions
from IPython.display import display, Markdown

# Load local models
default is zephyr-7b-beta and bge-base-en-v1.5

In [11]:
llm_args = load_local_models()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [12]:
!wget https://xdd.wisc.edu/dev_subsets/geoarchive_sample.zip
!unzip geoarchive_sample.zip

--2023-11-01 21:38:00--  https://xdd.wisc.edu/dev_subsets/geoarchive_sample.zip
Resolving xdd.wisc.edu (xdd.wisc.edu)... 128.104.103.121
Connecting to xdd.wisc.edu (xdd.wisc.edu)|128.104.103.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11013450 (11M) [application/zip]
Saving to: ‘geoarchive_sample.zip’

geoarchive_sample.z 100%[===================>]  10.50M  2.08MB/s    in 9.4s    

2023-11-01 21:38:10 (1.11 MB/s) - ‘geoarchive_sample.zip’ saved [11013450/11013450]

Archive:  geoarchive_sample.zip
  inflating: bibjson                 
   creating: raw_text/
  inflating: raw_text/620c1c1bad0e9c819b023e9b.txt  
  inflating: raw_text/620f2eacad0e9c819b10809e.txt  
  inflating: raw_text/620e6881ad0e9c819b0c36cc.txt  
  inflating: raw_text/620d1070ad0e9c819b058c05.txt  
  inflating: raw_text/620d2d57ad0e9c819b068363.txt  
  inflating: raw_text/620ec50aad0e9c819b0ed736.txt  
  inflating: raw_text/620ed91bad0e9c819b0f2b39.txt  
  inflating: raw_text/620d106bad

In [68]:
import factsheet_generator
importlib.reload(factsheet_generator)
from factsheet_generator import FactsheetGenerator, load_local_models

In [69]:
generator = factsheet_generator.FactsheetGenerator()

In [70]:
generator.init_vector_db() # initialize chromadb vector database

In [71]:
generator.use_local_model(llm_args, debug=True) # tell the generator to use the local models

In [19]:
generator.generate_vector_index(input_dir="raw_text") # generate embeddings for the text in the input directory

Parsing documents into nodes:   0%|          | 0/85 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/5881 [00:00<?, ?it/s]

In [20]:
strat_name = "Belle Fourche Formation"

In [79]:
questions = {
    "lithology": "What lithology does the {strat_name} consist of?",
    "time period": "What time period is the {strat_name} from?",
}

In [87]:
response_list = generator.generate_facts(strat_name, questions)

In [88]:
for category in response_list.keys():
    display(Markdown(f"{category}: {response_list[category].response}"))

lithology: The lithology of the Belle Fourche Formation consists of sandy and shaley mudstones.

time period: The time period of the Belle Fourche Formation is from the Middle Cretaceous.

In [89]:
response_list = generator.generate_facts(strat_name, questions, must_include="Belle Fourche", top_n=20) # force retrieved documents to contain the strat name

In [90]:
for category in response_list.keys():
    display(Markdown(f"{category}: {response_list[category].response}"))

lithology: The Belle Fourche Formation consists of sandy and shaley mudstones.

time period: The Belle Fourche Formation is from the Upper and Middle Cretaceous time period.

In [61]:
# response_list['location'].source_nodes[0].text

# Evaluation
Use GPT-4 to compare answers and check if retrieved top ranked sources are relevant to the query. The input data may not necessarily contain the relavent information.

In [83]:
answers = {
    "lithology": "sandstone, mudstone",
    "time period": "Cretaceous",
}

In [86]:
generator.generate_facts(strat_name, questions, eval=True, answers=answers)

,category,correct source,correct answer,LLM review
0,lithology,True,True,Context is relevant: True\nAnswer is correct: ...
1,time period,False,True,Context is relevant: False\nAnswer is correct:...


In [85]:
generator.generate_facts(strat_name, questions, must_include="Belle Fourche", top_n=20, eval=True, answers=answers)

,category,correct source,correct answer,LLM review
0,lithology,True,True,Context is relevant: True\nAnswer is correct: ...
1,time period,True,True,Context is relevant: True\nAnswer is correct: ...
